In [53]:
from pathlib import Path
from decode_fish.funcs.utils import *
import glob

def make_sweep_str(pfile, pars, group, resume=False, init=None, gpu='gpu_rtx', runfile='train.py'):
    
    base_string = f'bsub -gpu "num=1" -q {gpu} -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/{runfile}'   
    
    base_str = base_string + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        if k!= 'run_name':
            base_str += ' ' + k + f'={pars[k]}'
            name += k.split('.')[-1]+f':{pars[k]}x'
            name = name.replace('[', '')
            name = name.replace(']', '')
            name = name.replace(',', 'x')
    name = name[:-1]
    
    if 'run_name' in pars:
        name=pars['run_name']
    
    if resume:
        base_str += ' training.resume=True '
    if init is not None:
        base_str += f' data_path.model_init={init} '
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(pfile, name, group):
    
    base_str = base_string + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [179]:
# cfg.training.mic.par_grads.channel_facs = False
# cfg.training.mic.par_grads.channel_shifts = False
# cfg.training.mic.par_grads.theta_par = False
# cfg.training.mic.par_grads.psf_vol = False

In [186]:
group = 'sweep_mop_20'
pfile = 'MERFISH_mop_20'

variable_col = param_iter()
variable_col.add('data_path.image_path', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images150.tif', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images151.tif', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images47.tif')
par_prod = variable_col.param_product()

In [224]:
group = 'sweep_mop_21'
pfile = 'MERFISH_mop_21'

variable_col = param_iter()
# variable_col.add('training.schedule','null')
# variable_col.add('training.mic.freq',3)
variable_col.add('genm.microscope.norm', 'escort', 'none')
variable_col.add('training.z_sig_fac', 1.)

# variable_col.add('training.mic.mean_diff', False)
# variable_col.add('training.mic.enabled',False)
par_prod = variable_col.param_product()

In [131]:
group = 'MERFISH_sweep_gen_2'
pfile = 'MERFISH_sweep_gen_2'

variable_col = param_iter()
variable_col.add('genm.microscope.norm', 'none', 'escort')
variable_col.add('training.mic.par_grads.z_facs', True)
variable_col.add('training.mic.mean_diff', True)
variable_col.add('training.mic.par_grads.theta_par', False)
variable_col.add('training.mic.opt.lr', 0.00025, 0.0005)
par_prod = variable_col.param_product()

In [225]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group, resume=False, gpu='gpu_rtx') #, 
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_21 genm.microscope.norm=escort training.z_sig_fac=1.0 run_name=norm:escortxz_sig_fac:1.0 output.group=sweep_mop_21

bsub -gpu "num=1" -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_21 genm.microscope.norm=none training.z_sig_fac=1.0 run_name=norm:nonexz_sig_fac:1.0 output.group=sweep_mop_21



In [146]:
files[0][101:]

'norm:escortxz_facs:Truexmean_diff:Truexlr:0.0005/microscope.pkl'

In [121]:
group = 'gensweep_mop_1'
pfile = 'gensweep_mop_1'

variable_col = param_iter()
# variable_col.add('training.mic.par_grads.psf_vol', False)
# variable_col.add('training.mic.par_grads.channel_shifts', False)
# variable_col.add('training.mic.par_grads.color_shifts', False)
variable_col.add('genm.microscope.norm', 'none')
variable_col.add('training.mic.par_grads.z_facs', True)
variable_col.add('training.mic.mean_diff', True)
variable_col.add('training.mic.opt.lr', 0.0005, 0.001)
variable_col.add('genm.microscope.scale', 400)

par_prod = variable_col.param_product()

In [122]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group, resume=False, gpu='gpu_rtx', runfile='gentrain.py') #, 
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/gentrain.py +experiment=gensweep_mop_1 genm.microscope.norm=none training.mic.par_grads.z_facs=True training.mic.par_grads.theta_par=False training.mic.mean_diff=True training.mic.opt.lr=0.0005 genm.microscope.scale=400 run_name=norm:nonexz_facs:Truextheta_par:Falsexmean_diff:Truexlr:0.0005xscale:400 output.group=gensweep_mop_1

bsub -gpu "num=1" -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/gentrain.py +experiment=gensweep_mop_1 genm.microscope.norm=none training.mic.par_grads.z_facs=True training.mic.par_grads.theta_par=False training.mic.mean_diff=True training.mic.opt.lr=0.001 genm.microscope.scale=400 run_na